Open Play Shots - Expected Goal Model - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [3]:
model_version = 8
model_name = 'expected_goal_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [5]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

In [6]:
FEATURES = ModellingDataContract.open_goal_modelling_feature_list

In [7]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_open_goal

Load Data

In [8]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,0,1,107.0,32.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,True,False,False,True,False,False,True,False,False
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,113.0,33.0,80.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,False,False,True,False,False,True,True,False,False
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,119.0,39.0,80.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,True,False,False,True,False,False,True,False,False
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,125.0,45.0,80.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,False,False,True,True,False,False,True,False,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,137.0,45.0,92.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,True,False,False,False,False,True,True,False,False


In [9]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [10]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [11]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [12]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.4280142445311175, 0.42787418655097614, 0.4277966101694915)

In [13]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
14,114,possGain,behind,1011.0,2,2025,2032.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,54.0,35.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,1,0,50.0,52.0,-2.0,Kick,Loose Ball Get,Kick Inside 50 Result,Kick Into F50,54.0,49.0,41.0,-9.0,35.0,33.0,31.0,42.0,2032.0,2029.0,2027.0,2027.0,7.0,3.0,5.385165,24.0,35.0,42.438190,0.969723,55.561011,0.085774,4.914467,0.269657,15.450186,576.0,3.178054,False,False,True,False,False,False,True,True,False,False,0,0,0,1,0
16,141,centreBounce,behind,1250.0,3,901,912.0,Brisbane Lions,Brisbane Lions,Keidean Coleman,Keidean_Coleman,Kick,36.0,-31.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,1,0,57.0,71.0,-14.0,Kick,Hard Ball Get,Contest Target,Ground Kick,36.0,32.0,29.0,6.0,-31.0,-32.0,-31.0,-2.0,912.0,910.0,909.0,906.0,11.0,2.0,4.123106,42.0,31.0,52.201533,0.635838,36.430858,0.099014,5.673100,0.306285,17.548851,1764.0,3.737670,False,False,True,False,True,False,False,False,True,False,0,1,0,0,0
18,159,centreBounce,behind,1413.0,3,1532,1548.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Kick,64.0,-18.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,1,0,58.0,89.0,-31.0,Kick,Hard Ball Get Crumb,Spoil,Kick Inside 50 Result,64.0,59.0,56.0,56.0,-18.0,-19.0,-2.0,-2.0,1548.0,1545.0,1543.0,1541.0,16.0,3.0,5.099020,14.0,18.0,22.803509,0.909753,52.125016,0.175769,10.070821,0.628272,35.997348,196.0,2.639057,False,False,True,False,True,False,False,True,False,False,0,1,0,0,0
24,199,possGain,behind,1726.0,4,826,867.0,Sydney,Sydney,Errol Gulden,Errol_Gulden,Kick,62.0,29.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,0,1,71.0,113.0,-42.0,Kick,Handball Received,Handball,Free For,62.0,66.0,67.0,66.0,29.0,30.0,25.0,23.0,867.0,866.0,865.0,855.0,41.0,1.0,4.123106,16.0,29.0,33.120990,1.066630,61.113418,0.094225,5.398697,0.305720,17.516484,256.0,2.772589,False,False,True,False,True,False,False,True,False,False,0,0,0,1,0
27,222,throwIn,behind,1875.0,4,1446,1483.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Kick,50.0,-13.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,1,0,77.0,122.0,-45.0,Kick,Handball Received,Handball,Handball Received,50.0,49.0,51.0,51.0,-13.0,-15.0,-16.0,-16.0,1483.0,1483.0,1482.0,1481.0,37.0,0.0,2.236068,28.0,13.0,30.870698,0.434670,24.904769,0.190080,10.890793,0.624506,35.781575,784.0,3.332205,False,False,True,False,False,False,True,True,False,False,0,0,0,0,1


Preprocess Data

In [14]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [15]:
X_train_preproc = X_train[FEATURES]
X_cal_preproc = X_cal[FEATURES]
X_test_preproc = X_test[FEATURES]

In [16]:
X_train_preproc.shape, X_cal_preproc.shape, X_test_preproc.shape

((5897, 17), (1475, 17), (1844, 17))

In [17]:
X_train_preproc.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0
1,35.0,28.0,25.0,22.0,7.000000,47.010637,0.416065,0.125105,0,0,0,1,0,43.0,19.0,19.0,1.0
2,50.0,50.0,50.0,-8.0,4.000000,36.878178,0.708626,0.132764,0,0,0,1,0,28.0,24.0,21.0,1.0
3,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0
4,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0


In [18]:
X_test_preproc.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
14,54.0,49.0,41.0,-9.0,5.385165,42.438190,0.969723,0.085774,0,0,0,1,0,24.0,35.0,7.0,3.0
16,36.0,32.0,29.0,6.0,4.123106,52.201533,0.635838,0.099014,0,1,0,0,0,42.0,31.0,11.0,2.0
18,64.0,59.0,56.0,56.0,5.099020,22.803509,0.909753,0.175769,0,1,0,0,0,14.0,18.0,16.0,3.0
24,62.0,66.0,67.0,66.0,4.123106,33.120990,1.066630,0.094225,0,0,0,1,0,16.0,29.0,41.0,1.0
27,50.0,49.0,51.0,51.0,2.236068,30.870698,0.434670,0.190080,0,0,0,0,1,28.0,13.0,37.0,0.0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [19]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [20]:
xgb_tuner.training_data.head()

,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action
0,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0
1,35.0,28.0,25.0,22.0,7.000000,47.010637,0.416065,0.125105,0,0,0,1,0,43.0,19.0,19.0,1.0
2,50.0,50.0,50.0,-8.0,4.000000,36.878178,0.708626,0.132764,0,0,0,1,0,28.0,24.0,21.0,1.0
3,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0
4,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0


In [21]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 12:22:39,192] A new study created in memory with name: no-name-28ec3b21-1747-4703-bd55-8f8fac5eff0a
[I 2023-07-11 12:22:39,281] Trial 0 finished with value: 0.6349947474355155 and parameters: {'max_depth': 17, 'min_child_weight': 15, 'eta': 0.4287739697812185, 'gamma': 0.051345749303974966, 'lambda': 1.233274047910685, 'alpha': 0.2416353607185702, 'subsample': 0.7331118456988579, 'colsample_bytree': 0.7937751926476067}. Best is trial 0 with value: 0.6349947474355155.
[I 2023-07-11 12:22:39,315] Trial 1 finished with value: 0.636859401341928 and parameters: {'max_depth': 15, 'min_child_weight': 37, 'eta': 0.119999858213984, 'gamma': 0.01374718138566586, 'lambda': 0.20516514666527466, 'alpha': 0.0209067434187948, 'subsample': 0.8880567727214386, 'colsample_bytree': 0.5336446620401833}. Best is trial 0 with value: 0.6349947474355155.
[I 2023-07-11 12:22:39,380] Trial 2 finished with value: 0.7237748733986267 and parameters: {'max_depth': 16, 'min_child_weight': 7, 'eta': 0.7

Number of finished trials:  1000
Best trial:
  Value: 0.6040921222210633
  Params: 
    max_depth: 10
    min_child_weight: 9
    eta: 0.38622997062392855
    gamma: 0.006820072981607029
    lambda: 0.24591923776543373
    alpha: 3.7417152243757386
    subsample: 0.31620738967797807
    colsample_bytree: 0.7036755416507048


In [22]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 10,
 'min_child_weight': 9,
 'eta': 0.38622997062392855,
 'gamma': 0.006820072981607029,
 'lambda': 0.24591923776543373,
 'alpha': 3.7417152243757386,
 'subsample': 0.31620738967797807,
 'colsample_bytree': 0.7036755416507048}

Training Model - SuperXGBClassifier class for training and predictions

In [23]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [24]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [25]:
super_xgb.fit()

[0]	validation_0-logloss:0.65670	validation_1-logloss:0.65661
[1]	validation_0-logloss:0.63628	validation_1-logloss:0.63916
[2]	validation_0-logloss:0.62663	validation_1-logloss:0.63224
[3]	validation_0-logloss:0.62032	validation_1-logloss:0.62805
[4]	validation_0-logloss:0.61475	validation_1-logloss:0.62427
[5]	validation_0-logloss:0.61180	validation_1-logloss:0.62359
[6]	validation_0-logloss:0.60689	validation_1-logloss:0.62115
[7]	validation_0-logloss:0.60486	validation_1-logloss:0.61899
[8]	validation_0-logloss:0.60286	validation_1-logloss:0.61954
[9]	validation_0-logloss:0.60062	validation_1-logloss:0.62123
[10]	validation_0-logloss:0.59830	validation_1-logloss:0.62107
[11]	validation_0-logloss:0.59686	validation_1-logloss:0.62147
[12]	validation_0-logloss:0.59304	validation_1-logloss:0.62269
[13]	validation_0-logloss:0.59203	validation_1-logloss:0.62214
[14]	validation_0-logloss:0.59150	validation_1-logloss:0.62198
[15]	validation_0-logloss:0.58959	validation_1-logloss:0.62289
[1

In [26]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7036755416507048, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.006820072981607029, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.386...
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=9, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_Since_Last_Action': 1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [27]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'x1',
 'x2',
 'x3',
 'Distance_Since_Last_Action',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'ballUp',
 'centreBounce',
 'kickIn',
 'possGain',
 'throwIn',
 'Distance_to_Right_Goal_x',
 'Distance_to_Middle_y',
 'Chain_Duration',
 'Time_Since_Last_Action']

In [28]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [29]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [30]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [31]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [34]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.43407786, 0.4280142445311175, 0.42865396234099584)

In [35]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.42767638, 0.42787418655097614, 0.42231037540873617)

Export model

In [36]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [37]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,1,11.0,1,26.0,18.0,11.0,6.0,8.246211,56.080300,0.383818,0.106164,0,1,0,0,0,52.0,21.0,26.0,1.0,0,0.324383,0.318738
1,202101_BrisbaneLions_Sydney,6,61.0,1,35.0,28.0,25.0,22.0,7.000000,47.010637,0.416065,0.125105,0,0,0,1,0,43.0,19.0,19.0,1.0,0,0.317634,0.313528
2,202101_BrisbaneLions_Sydney,10,110.0,1,50.0,50.0,50.0,-8.0,4.000000,36.878178,0.708626,0.132764,0,0,0,1,0,28.0,24.0,21.0,1.0,0,0.337968,0.329366
3,202101_BrisbaneLions_Sydney,21,222.0,1,34.0,19.0,10.0,7.0,15.132746,46.486557,0.328553,0.130930,0,0,0,1,0,44.0,15.0,13.0,2.0,0,0.358354,0.345683
4,202101_BrisbaneLions_Sydney,38,392.0,0,40.0,36.0,34.0,32.0,4.123106,39.560081,0.281772,0.156423,0,0,0,1,0,38.0,11.0,5.0,3.0,0,0.401741,0.382008


In [38]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,x1,x2,x3,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,Distance_to_Right_Goal_x,Distance_to_Middle_y,Chain_Duration,Time_Since_Last_Action,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,114,1011.0,0,54.0,49.0,41.0,-9.0,5.385165,42.438190,0.969723,0.085774,0,0,0,1,0,24.0,35.0,7.0,3.0,0,0.250729,0.264046
1,202101_BrisbaneLions_Sydney,141,1250.0,0,36.0,32.0,29.0,6.0,4.123106,52.201533,0.635838,0.099014,0,1,0,0,0,42.0,31.0,11.0,2.0,0,0.294986,0.296359
2,202101_BrisbaneLions_Sydney,159,1413.0,0,64.0,59.0,56.0,56.0,5.099020,22.803509,0.909753,0.175769,0,1,0,0,0,14.0,18.0,16.0,3.0,0,0.319459,0.314932
3,202101_BrisbaneLions_Sydney,199,1726.0,0,62.0,66.0,67.0,66.0,4.123106,33.120990,1.066630,0.094225,0,0,0,1,0,16.0,29.0,41.0,1.0,0,0.290309,0.292870
4,202101_BrisbaneLions_Sydney,222,1875.0,0,50.0,49.0,51.0,51.0,2.236068,30.870698,0.434670,0.190080,0,0,0,0,1,28.0,13.0,37.0,0.0,0,0.455364,0.430221


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")